<a href="https://colab.research.google.com/github/heber-augusto/albert-demo-teste/blob/main/notebooks/finetune_tradutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ajuste fino modelo de tradução (en -> pt)
Heber Augusto Scachetti

# Tradução Automática de Textos Inglês-Português com Transformer


In [1]:
!pip install datasets transformers[sentencepiece] sacrebleu
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00


In [3]:

import pandas as pd
from datasets import DatasetDict, Dataset, Features, features, Translation
from datasets import load_from_disk
import os
import datasets
import random

# Leitura do dataset ParaCrawl99
traducoes_en_pt = pd.read_pickle(r'https://github.com/unicamp-dl/Lite-T5-Translation/raw/master/ParaCrawl99k/ParaCrawl99K_EnPt_PCrawlGoogleT.pkl')
df_traducoes_en_pt = pd.DataFrame(traducoes_en_pt, columns=['en','pt'])

# Função para converter o dataframe pandas em um Dataset do Hugging Face
# essa função foi criada com o apoio do chatgpt (não encontrei exemplos na documentação)
def pandas_to_huggingface_dataset(df):
    dataset = Dataset.from_pandas(df)
    dataset = dataset.map(lambda example: {'translation': {'en': example['en'], 'pt': example['pt']}}, remove_columns=['en', 'pt'])

    # Definir o esquema do Dataset
    dataset = dataset.cast(Features({'translation': Translation(languages=('en', 'pt'))}))

    return dataset

# salva arquivo de traducoes en_pt
hf_dataset = pandas_to_huggingface_dataset(df_traducoes_en_pt)
input_dir = r'/content/input'
hf_dataset.save_to_disk(os.path.join(input_dir,"traducoes_en_pt.hf"))

# #1. Separação em treino, validação e teste
# Separe o conjunto de dados ParaCrawl99k em 80% para treino, 10% para validação e 10% para teste.
# Todos os experimentos intermediários devem ser avaliados no conjunto de validação e, apenas ao final, reportar
# o resultado do melhor modelo no conjunto de teste.

train_test_dict = hf_dataset.train_test_split(test_size=0.2)
train = train_test_dict['train']
test_and_val = train_test_dict['test']
train_test_dict = test_and_val.train_test_split(test_size=0.5)
test = train_test_dict['train']
val = train_test_dict['test']

ds = DatasetDict({
    'train': train,
    'test': test,
    'valid': val})


def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset)
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    return df


df_to_display = show_random_elements(ds["train"])

for row in df_to_display['translation']:
    print(row)

Map:   0%|          | 0/99087 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/99087 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/99087 [00:00<?, ? examples/s]

{'en': 'Mean body mass index (BMI) was 28.94 ± 6.06 kg/m2 in the patients with calcification and 26.84 ± 4.11 kg/m2 in the patients without the condition.', 'pt': 'O índice de massa corporal médio (IMC) foi de 28,94 ± 6,06 kg / m2 nos pacientes com calcificação e 26,84 ± 4,11 kg / m2 nos pacientes sem a condição.'}
{'en': 'The radical obedience of Jesus even to his death on the cross (cf. Phil 2:8) completely unmasks this sin.', 'pt': 'A radical obediência de Jesus até sua morte na cruz (cf. Fl 2, 8) desmascara completamente esse pecado.'}
{'en': 'On December 13rd, 2019, five units KX-3000X juice vending machines customized by KONMAX according to customer requirements were delivered to the Port of Liverpool, UK and New Delhi, India on time.', 'pt': 'Em 13 de dezembro de 2019, cinco unidades de máquinas de venda automática de suco KX-3000X personalizadas pela KONMAX de acordo com as necessidades do cliente foram entregues no Porto de Liverpool, Reino Unido e Nova Délhi, na Índia, pontua

In [7]:
import pandas as pd
from datasets import DatasetDict, Dataset, Features, Translation
from datasets import load_from_disk
import os
from transformers import T5Tokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from keras.optimizers import Adam
import evaluate




def preprocess_function(examples, tokenizer):
    source_lang = "en"
    target_lang = "pt"
    prefix = "translate English to Portuguese: "
    max_length=64

    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(
        inputs,
        padding="max_length",
        max_length=max_length,
        truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            padding="max_length",
            max_length=max_length,
            truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


def get_output_texts(model, tokenizer, tf_set, max_length):
    output_texts = []
    for batch_idx, batch in enumerate(tf_set):
        print(f'generating output: {batch_idx}')
        source_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        target_ids = batch['labels']
        outputs = model.generate(
            source_ids,
            attention_mask=attention_mask,
            do_sample=False,
            max_length=max_length)
        text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        output_texts.extend(text)

    return output_texts

def load_tokenizer():
    """
    Carrega o modelo de generative AI a ser fine-tuned.

    Retorna:
        model: Modelo carregado
    """
    model_name = 'unicamp-dl/ptt5-small-portuguese-vocab'
    tokenizer = T5Tokenizer.from_pretrained(model_name)

    #with tokenizer.as_target_tokenizer():
    #    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

    return tokenizer



def load_generative_model():
    """
    Carrega o modelo de generative AI a ser fine-tuned.

    Retorna:
        model: Modelo carregado
    """

    model_name = 'unicamp-dl/ptt5-small-portuguese-vocab'
    model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

    return model


def fine_tune_model(model, input_data, output_dir):
    """
    Realiza o fine-tuning do modelo de generative AI.

    Args:
        model: Modelo de generative AI carregado
        input_data (str): Caminho para os dados de entrada
        output_dir (str): Caminho para o diretório de saída
    """

    max_length=64
    batch_size = 64


    hf_dataset = load_from_disk(input_data)
    train_test_dict = hf_dataset.train_test_split(test_size=0.2)
    train = train_test_dict['train']
    test_and_val = train_test_dict['test']
    train_test_dict = test_and_val.train_test_split(test_size=0.5)
    test = train_test_dict['train']
    val = train_test_dict['test']

    ds = DatasetDict({
        'train': train,
        'test': test,
        'valid': val})

    tokenizer = load_tokenizer()
    tokenized_texts = ds.map(
        preprocess_function,
        fn_kwargs={"tokenizer": tokenizer,},
        batched=True)


    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        return_tensors="tf",
        padding='max_length',
        max_length=max_length)



    tf_train_set = tokenized_texts["train"].to_tf_dataset(
        columns=["attention_mask", "input_ids", "labels"],
        shuffle=True,
        batch_size=batch_size,
        collate_fn=data_collator,
    )

    tf_test_set = tokenized_texts["test"].to_tf_dataset(
        columns=["attention_mask", "input_ids", "labels"],
        shuffle=False,
        batch_size=batch_size,
        collate_fn=data_collator,
    )

    tf_val_set = tokenized_texts["valid"].to_tf_dataset(
        columns=["attention_mask", "input_ids", "labels"],
        shuffle=False,
        batch_size=batch_size,
        collate_fn=data_collator,
    )

    # output_texts = get_output_texts(
    #     model,
    #     tokenizer,
    #     tf_val_set,
    #     max_length)

    # ref_pt = []
    # for elem in ds['valid']:
    #     ref_pt.append(elem['translation']['pt'])
    # bleu = evaluate.load("bleu")
    # results = bleu.compute(
    #     predictions=output_texts,
    #     references=ref_pt)

    # print(results)


    optimizer = Adam(learning_rate=0.01)
    model.compile(optimizer=optimizer)
    model.fit(
        x=tf_train_set,
        validation_data=tf_val_set,
        epochs=10)

    model.save_pretrained(os.path.join(output_dir, 'saved_model'))


    return model

def main():

    # Carregue o modelo
    generative_model = load_generative_model()

    # Defina o caminho para os dados de entrada
    input_data = "./input/traducoes_en_pt.hf"  # Altere conforme necessário

    # Defina o diretório de saída
    output_dir = "./output"  # Altere conforme necessário

    # Realize o fine-tuning
    return fine_tune_model(generative_model, input_data, output_dir)


generative_model = main()


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at unicamp-dl/ptt5-small-portuguese-vocab.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Map:   0%|          | 0/79269 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/9909 [00:00<?, ? examples/s]

Map:   0%|          | 0/9909 [00:00<?, ? examples/s]

Epoch 1/10
1239/1239 [==============================] - 790s 624ms/step - loss: 1.3591 - val_loss: 0.8459
Epoch 2/10
1239/1239 [==============================] - 771s 622ms/step - loss: 0.8892 - val_loss: 0.7370
Epoch 3/10
1239/1239 [==============================] - 761s 615ms/step - loss: 0.7904 - val_loss: 0.6853
Epoch 4/10
1239/1239 [==============================] - 769s 621ms/step - loss: 0.7284 - val_loss: 0.6545
Epoch 5/10
1239/1239 [==============================] - 770s 621ms/step - loss: 0.9359 - val_loss: 0.6980
Epoch 6/10
1239/1239 [==============================] - 761s 614ms/step - loss: 0.7164 - val_loss: 0.6499
Epoch 7/10
1239/1239 [==============================] - 770s 622ms/step - loss: 0.6747 - val_loss: 0.6403
Epoch 8/10
1239/1239 [==============================] - 769s 621ms/step - loss: 0.6533 - val_loss: 0.6428
Epoch 9/10
1239/1239 [==============================] - 759s 613ms/step - loss: 0.6821 - val_loss: 0.6306
Epoch 10/10
1239/1239 [=======================